In [1]:
import tensorflow
import tensorflow.compat.v1 as tf
import numpy as np

![](picture/CNN.png)
![](picture/CNN2.png)
![](picture/离散卷积.png)
![](picture/卷积.png)
![](picture/卷积2.png)
![](picture/卷积3.png)
![](picture/卷积四.png)

In [6]:
def convld(x,w,p,s=1):
    w_rot=np.array(w[::-1])
    x_padded=np.array(x)
    if p>0:
        zero_pad=np.zeros(shape=p)
        x_padded=np.concatenate([zero_pad,x_padded,zero_pad])
    res=[]
    for i in range(0,int(len(x)/s),s):
        res.append(np.sum(x_padded[i:i+w_rot.shape[0]]*w_rot))
    return np.array(res)

In [8]:
x=[1,3,2,4,5,6,1,3]
w=[1,0,3,1,2]
print(convld(x,w,p=2,s=1))
print(np.convolve(x,w,mode='same'))

[ 5. 14. 16. 26. 24. 34. 19. 22.]
[ 5 14 16 26 24 34 19 22]


![](picture/卷积5.png)
![](picture/卷积6.png)

In [11]:
def conv2d(x,w,p=(0,0),s=(1,1)):
    w_rot=np.array(w)[::-1,::-1]
    x_orig=np.array(x)
    n1=x_orig.shape[0]+2*p[0]
    n2=x_orig.shape[1]+2*p[1]
    x_padded=np.zeros(shape=(n1,n2))
    x_padded[p[0]:p[0]+x_orig.shape[0],p[1]:p[1]+x_orig.shape[1]]=x_orig
    res=[]
    for i in range(0,int((x_padded.shape[0]-w_rot.shape[0])/s[0]+1),s[0]):
        res.append([])
        for j in range(0,int((x_padded.shape[1]-w_rot.shape[1])/s[1]+1),s[1]):
            x_sub=x_padded[i:i+w_rot.shape[0],j:j+w_rot.shape[1]]
            res[-1].append(np.sum(x_sub*w_rot))
    return (np.array(res))

In [12]:
x=[[1,2,3,4],[5,6,1,3],[1,6,0,3],[3,4,3,2]]
w=[[1,0,3],[1,2,1],[0,1,1]]
print(conv2d(x,w,p=(1,1,),s=(1,1)))

[[10. 24. 33. 14.]
 [23. 24. 37. 14.]
 [17. 36. 30. 19.]
 [11. 21. 18. 10.]]


In [13]:
import scipy.signal
print(scipy.signal.convolve2d(x,w,mode='same'))

[[10 24 33 14]
 [23 24 37 14]
 [17 36 30 19]
 [11 21 18 10]]


![](picture/子采样.png)
![](picture/池化.png)
![](picture/卷积神经网络.png)

In [46]:
#处理多个输入维度的数据
from skimage import io
import matplotlib.pyplot as plt
img=io.imread('data/example-image.png')

In [51]:
print(img.dtype)
print(img.shape)
print(img[100:102,100:102,0:3])

uint8
(252, 221, 4)
[[[179 134 110]
  [182 136 112]]

 [[180 135 111]
  [182 137 113]]]


![](picture/卷积7.png)
![](picture/卷积8.png)
![](picture/正则化淘汰神经网络.png)
![](picture/正则化淘汰神经网络2.png)

![](picture/卷积9.png)

In [4]:
mnist=np.load("data/mnist_scaled.npz")#标准化后的数据
X_train,y_train,X_test,y_test=[mnist[s] for s in mnist]

In [5]:
X_train,y_train,X_valid,y_valid=X_train[:50000,:],y_train[:50000],X_train[50000:,:],y_train[50000:]

In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(50000, 784)
(50000,)
(10000, 784)
(10000,)
(10000, 784)
(10000,)


In [8]:
#定义一个产生小批量数据的函数

def batch_generator(X,y,batch_size=64,shuffle=False,random_seed=None):
    idx=np.arange(y.shape[0])
    if shuffle:
        rng=np.random.RandomState(random_seed)
        rng.shuffle(idx)
        X=X[idx]
        y=y[idx]
    for i in range(0,X.shape[0],batch_size):
        yield X[i:i+batch_size,:],y[i:i+batch_size]

In [53]:
#使用底层API实现CNN
def conv_layer(input_tensor,name,kernel_size,n_output_channels,padding_mode="SAME",strides=(1,1,1,1)):
    with tf.variable_scope(name):
        ## get n_channels

        input_shape=input_tensor.get_shape().as_list()
        n_input_channels=input_shape[-1]
        weights_shape=list(kernel_size)+[n_input_channels,n_output_channels]
        weights=tf.get_variable(name='weights',shape=weights_shape)
        print(weights)
        biases=tf.get_variable(name='biases',initializer=tf.zeros(shape=[n_output_channels]))
        print(biases)
        conv=tf.nn.conv2d(input=input_tensor,
                          filter=weights,
                          strides=strides,
                          padding=padding_mode)
        print(conv)
        conv=tf.nn.bias_add(conv,biases,name='net_pre-activation')
        print(conv)
        conv=tf.nn.relu(conv,name='activation')
        print(conv)
        return conv

In [21]:
g=tf.Graph()
with g.as_default():
    x=tf.placeholder(tf.float32,shape=[None,28,28,1])
    conv_layer(x,name='convtest',
               kernel_size=(3,3),
               n_output_channels=32)


<tf.Variable 'convtest/weights:0' shape=(3, 3, 1, 32) dtype=float32>
<tf.Variable 'convtest/biases:0' shape=(32,) dtype=float32>
Tensor("convtest/Conv2D:0", shape=(None, 28, 28, 32), dtype=float32)
Tensor("convtest/net_pre-activation:0", shape=(None, 28, 28, 32), dtype=float32)
Tensor("convtest/activation:0", shape=(None, 28, 28, 32), dtype=float32)


In [22]:
del g,x

In [30]:
#定义全连接层
def fc_layer(input_tensor,name,
             n_output_units,activation_fn=None):
    with tf.variable_scope(name):
        input_shape=input_tensor.get_shape().as_list()[1:]
        n_input_units=np.prod(input_shape)
        if len(input_shape)>1:
            input_tensor=tf.reshape(input_tensor,shape=(-1,n_input_units))
        weights_shape=[n_input_units,n_output_units]
        weights=tf.get_variable(name='weights',shape=weights_shape)
        print(weights)
        biases=tf.get_variable(name='biases',initializer=tf.zeros(shape=[n_output_units]))
        print(biases)
        layer=tf.matmul(input_tensor,weights)
        print(layer)
        layer=tf.nn.bias_add(layer,biases,name='net_pre-activation')
        print(layer)
        if activation_fn==None:
            return layer
        layer=activation_fn(layer,name='activation')
        print(layer)
        return layer

In [31]:
g=tf.Graph()
with g.as_default():
    x=tf.placeholder(tf.float32,
                     name='x',shape=[None,28,28,1])
    fc_layer(x,name='fctest',n_output_units=32,
             activation_fn=tf.nn.relu)

<tf.Variable 'fctest/weights:0' shape=(784, 32) dtype=float32>
<tf.Variable 'fctest/biases:0' shape=(32,) dtype=float32>
Tensor("fctest/MatMul:0", shape=(None, 32), dtype=float32)
Tensor("fctest/net_pre-activation:0", shape=(None, 32), dtype=float32)
Tensor("fctest/activation:0", shape=(None, 32), dtype=float32)


In [32]:
del g,x

In [44]:
def build_cnn(learning_rate=1e-4):
    tf_x=tf.placeholder(tf.float32,shape=[None,784],
                        name='tf_x')
    tf_y=tf.placeholder(tf.int32,shape=[None],
                        name='tf_y')
    tf_x_image=tf.reshape(tf_x,shape=[-1,28,28,1],name='tf_x_reshaped')
    tf_y_onehot=tf.one_hot(indices=tf_y,depth=10,dtype=tf.float32,
                           name='tf_y_onehot')
    print('\nBuilding 1st layer')
    h1=conv_layer(tf_x_image,name='conv_1',
                  kernel_size=(5,5),
                  padding_mode='VALID',
                  n_output_channels=32)
    ##maxpooling
    h1_pool=tf.nn.max_pool(h1,
                           ksize=[1,2,2,1],
                           strides=[1,2,2,1],
                           padding='SAME')
    ##第二层卷积层
    print('\nBuilding 2nd layer:')
    h2=conv_layer(h1_pool,name='conv2',
                  kernel_size=(5,5),
                  padding_mode="VALID",
                  n_output_channels=64)
    h2_pool=tf.nn.max_pool(h2,ksize=[1,2,2,1],
                           strides=[1,2,2,1],
                           padding="SAME")
    ##第三层全连接层
    print('\nBuilding 3rd layer:')
    h3=fc_layer(h2_pool,name='fc_3',
                n_output_units=1024,
                activation_fn=tf.nn.relu)
    ##Drop out
    keep_prob=tf.placeholder(tf.float32,name='fc_keep_prob')
    h3_drop=tf.nn.dropout(h3,keep_prob=keep_prob,
                          name='dropout_layer')
    ##全连接层
    print('\nBuilding 4th layer:')
    h4=fc_layer(h3_drop,name='fc_4',
                n_output_units=10,
                activation_fn=None)
    ##Predictions
    predictions={'probabilities':tf.nn.softmax(h4,name='probabilities'),
                 'labels':tf.cast(tf.argmax(h4,axis=1),tf.int32,
                                  name='labels')}
    cross_entropy_loss=tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(logits=h4,labels=tf_y_onehot),
        name='cross_entropy_loss'
    )
    optimizer=tf.train.AdamOptimizer(learning_rate)
    optimizer=optimizer.minimize(cross_entropy_loss,name='train_op')
    correct_predictions=tf.equal(predictions['labels'],
                                 tf_y,name='correct_preds')
    accuracy=tf.reduce_mean(
        tf.cast(correct_predictions,tf.float32),
        name='accuracy'
    )

In [45]:
def save(saver, sess, epoch, path='./model/'):
    if not os.path.isdir(path):
        os.makedirs(path)
    print('Saving model in %s' % path)
    saver.save(sess, os.path.join(path,'cnn-model.ckpt'),
               global_step=epoch)


def load(saver, sess, path, epoch):
    print('Loading model from %s' % path)
    saver.restore(sess, os.path.join(
            path, 'cnn-model.ckpt-%d' % epoch))


def train(sess, training_set, validation_set=None,
          initialize=True, epochs=20, shuffle=True,
          dropout=0.5, random_seed=None):

    X_data = np.array(training_set[0])
    y_data = np.array(training_set[1])
    training_loss = []

    ## initialize variables
    if initialize:
        sess.run(tf.global_variables_initializer())

    np.random.seed(random_seed) # for shuflling in batch_generator
    for epoch in range(1, epochs+1):
        batch_gen = batch_generator(
                        X_data, y_data,
                        shuffle=shuffle)
        avg_loss = 0.0
        for i,(batch_x,batch_y) in enumerate(batch_gen):
            feed = {'tf_x:0': batch_x,
                    'tf_y:0': batch_y,
                    'fc_keep_prob:0': dropout}
            loss, _ = sess.run(
                    ['cross_entropy_loss:0', 'train_op'],
                    feed_dict=feed)
            avg_loss += loss

        training_loss.append(avg_loss / (i+1))
        print('Epoch %02d Training Avg. Loss: %7.3f' % (
            epoch, avg_loss), end=' ')
        if validation_set is not None:
            feed = {'tf_x:0': validation_set[0],
                    'tf_y:0': validation_set[1],
                    'fc_keep_prob:0':1.0}
            valid_acc = sess.run('accuracy:0', feed_dict=feed)
            print(' Validation Acc: %7.3f' % valid_acc)
        else:
            print()


def predict(sess, X_test, return_proba=False):
    feed = {'tf_x:0': X_test,
            'fc_keep_prob:0': 1.0}
    if return_proba:
        return sess.run('probabilities:0', feed_dict=feed)
    else:
        return sess.run('labels:0', feed_dict=feed)

In [46]:
## create a graph
random_seed = 123

np.random.seed(random_seed)
g = tf.Graph()
with g.as_default():
    tf.set_random_seed(random_seed)
    ## build the graph
    build_cnn()

    ## saver:
    saver = tf.train.Saver()


Building 1st layer
<tf.Variable 'conv_1/weights:0' shape=(5, 5, 1, 32) dtype=float32>
<tf.Variable 'conv_1/biases:0' shape=(32,) dtype=float32>
Tensor("conv_1/Conv2D:0", shape=(None, 24, 24, 32), dtype=float32)
Tensor("conv_1/net_pre-activation:0", shape=(None, 24, 24, 32), dtype=float32)
Tensor("conv_1/activation:0", shape=(None, 24, 24, 32), dtype=float32)

Building 2nd layer:
<tf.Variable 'conv2/weights:0' shape=(5, 5, 32, 64) dtype=float32>
<tf.Variable 'conv2/biases:0' shape=(64,) dtype=float32>
Tensor("conv2/Conv2D:0", shape=(None, 8, 8, 64), dtype=float32)
Tensor("conv2/net_pre-activation:0", shape=(None, 8, 8, 64), dtype=float32)
Tensor("conv2/activation:0", shape=(None, 8, 8, 64), dtype=float32)

Building 3rd layer:
<tf.Variable 'fc_3/weights:0' shape=(1024, 1024) dtype=float32>
<tf.Variable 'fc_3/biases:0' shape=(1024,) dtype=float32>
Tensor("fc_3/MatMul:0", shape=(None, 1024), dtype=float32)
Tensor("fc_3/net_pre-activation:0", shape=(None, 1024), dtype=float32)
Tensor("fc_3

In [56]:
with tf.Session(graph=g) as sess:
    train(sess,
          training_set=(X_train, y_train),
          validation_set=(X_valid, y_valid),
          initialize=True,
          random_seed=123)
    save(saver, sess, epoch=20)

Epoch 01 Training Avg. Loss: 307.847  Validation Acc:   0.972
Epoch 02 Training Avg. Loss:  79.140  Validation Acc:   0.982
Epoch 03 Training Avg. Loss:  53.798  Validation Acc:   0.985
Epoch 04 Training Avg. Loss:  41.890  Validation Acc:   0.987
Epoch 05 Training Avg. Loss:  34.263  Validation Acc:   0.989


KeyboardInterrupt: 

In [ ]:
g2 = tf.Graph()
with g2.as_default():
    tf.set_random_seed(random_seed)
    ## build the graph
    build_cnn()

    ## saver:
    saver = tf.train.Saver()

## create a new session
## and restore the model
with tf.Session(graph=g2) as sess:
    load(saver, sess,
         epoch=20, path='./model/')

    preds = predict(sess, X_test_centered,
                    return_proba=False)

    print('Test Accuracy: %.3f%%' % (100*
                np.sum(preds == y_test)/len(y_test)))

In [ ]:
np.set_printoptions(precision=2, suppress=True)

with tf.Session(graph=g2) as sess:
    load(saver, sess,
         epoch=20, path='./model/')

    print(predict(sess, X_test_centered[:10],
              return_proba=False))

    print(predict(sess, X_test_centered[:10],
                  return_proba=True))

In [ ]:
with tf.Session(graph=g2) as sess:
    load(saver, sess,
         epoch=20, path='./model/')

    train(sess,
          training_set=(X_train_centered, y_train),
          validation_set=(X_valid_centered, y_valid),
          initialize=False,
          epochs=20,
          random_seed=123)

    save(saver, sess, epoch=40, path='./model/')

    preds = predict(sess, X_test_centered,
                    return_proba=False)

    print('Test Accuracy: %.3f%%' % (100*
                np.sum(preds == y_test)/len(y_test)))

### 使用tensorflow的layers的API来实现CNN

In [2]:
import os
class ConvNN(object):
    def __init__(self, batchsize=64,
                 epochs=20, learning_rate=1e-4,
                 dropout_rate=0.5,
                 shuffle=True, random_seed=None):
        np.random.seed(random_seed)
        self.batchsize = batchsize#批量样本数量
        self.epochs = epochs#迭代次数
        self.learning_rate = learning_rate#学习率
        self.dropout_rate = dropout_rate#删除率
        self.shuffle = shuffle#重排序

        g = tf.Graph()
        with g.as_default():
            ## set random-seed:
            tf.set_random_seed(random_seed)

            ## build the network:
            self.build()

            ## initializer
            self.init_op =                 tf.global_variables_initializer()

            ## saver
            self.saver = tf.train.Saver()

        ## create a session
        self.sess = tf.Session(graph=g)

    def build(self):

        ## Placeholders for X and y:
        tf_x = tf.placeholder(tf.float32,
                              shape=[None, 784],
                              name='tf_x')
        tf_y = tf.placeholder(tf.int32,
                              shape=[None],
                              name='tf_y')
        is_train = tf.placeholder(tf.bool,
                              shape=(),
                              name='is_train')

        ## reshape x to a 4D tensor:
        ##  [batchsize, width, height, 1]
        tf_x_image = tf.reshape(tf_x, shape=[-1, 28, 28, 1],
                              name='input_x_2dimages')
        ## One-hot encoding:
        tf_y_onehot = tf.one_hot(indices=tf_y, depth=10,
                              dtype=tf.float32,
                              name='input_y_onehot')

        ## 1st layer: Conv_1
        h1 = tf.layers.conv2d(tf_x_image,
                              kernel_size=(5, 5),
                              filters=32,
                              activation=tf.nn.relu)
        ## MaxPooling
        h1_pool = tf.layers.max_pooling2d(h1,
                              pool_size=(2, 2),
                              strides=(2, 2))
        ## 2n layer: Conv_2
        h2 = tf.layers.conv2d(h1_pool, kernel_size=(5,5),
                              filters=64,
                              activation=tf.nn.relu)
        ## MaxPooling
        h2_pool = tf.layers.max_pooling2d(h2,
                              pool_size=(2, 2),
                              strides=(2, 2))

        ## 3rd layer: Fully Connected
        input_shape = h2_pool.get_shape().as_list()
        n_input_units = np.prod(input_shape[1:])
        h2_pool_flat = tf.reshape(h2_pool,
                              shape=[-1, n_input_units])
        h3 = tf.layers.dense(h2_pool_flat, 1024,
                              activation=tf.nn.relu)

        ## Dropout
        h3_drop = tf.layers.dropout(h3,
                              rate=self.dropout_rate,
                              training=is_train)

        ## 4th layer: Fully Connected (linear activation)
        h4 = tf.layers.dense(h3_drop, 10,
                              activation=None)

        ## Prediction
        predictions = {
            'probabilities': tf.nn.softmax(h4,
                              name='probabilities'),
            'labels': tf.cast(tf.argmax(h4, axis=1),
                              tf.int32, name='labels')}

        ## Loss Function and Optimization
        cross_entropy_loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=h4, labels=tf_y_onehot),
            name='cross_entropy_loss')

        ## Optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        optimizer = optimizer.minimize(cross_entropy_loss,
                                       name='train_op')

        ## Finding accuracy
        correct_predictions = tf.equal(
            predictions['labels'],
            tf_y, name='correct_preds')

        accuracy = tf.reduce_mean(
            tf.cast(correct_predictions, tf.float32),
            name='accuracy')

    def save(self, epoch, path='./tflayers-model/'):
        if not os.path.isdir(path):
            os.makedirs(path)
        print('Saving model in %s' % path)
        self.saver.save(self.sess,
                        os.path.join(path, 'model.ckpt'),
                        global_step=epoch)

    def load(self, epoch, path):
        print('Loading model from %s' % path)
        self.saver.restore(self.sess,
             os.path.join(path, 'model.ckpt-%d' % epoch))

    def train(self, training_set,
              validation_set=None,
              initialize=True):
        ## initialize variables
        if initialize:
            self.sess.run(self.init_op)

        self.train_cost_ = []
        X_data = np.array(training_set[0])
        y_data = np.array(training_set[1])

        for epoch in range(1, self.epochs + 1):
            batch_gen =                 batch_generator(X_data, y_data,
                                 shuffle=self.shuffle)
            avg_loss = 0.0
            for i, (batch_x,batch_y) in                 enumerate(batch_gen):
                feed = {'tf_x:0': batch_x,
                        'tf_y:0': batch_y,
                        'is_train:0': True} ## for dropout
                loss, _ = self.sess.run(
                        ['cross_entropy_loss:0', 'train_op'],
                        feed_dict=feed)
                avg_loss += loss

            print('Epoch %02d: Training Avg. Loss: '
                  '%7.3f' % (epoch, avg_loss), end=' ')
            if validation_set is not None:
                feed = {'tf_x:0': batch_x,
                        'tf_y:0': batch_y,
                        'is_train:0': False} ## for dropout
                valid_acc = self.sess.run('accuracy:0',
                                          feed_dict=feed)
                print('Validation Acc: %7.3f' % valid_acc)
            else:
                print()

    def predict(self, X_test, return_proba = False):
        feed = {'tf_x:0': X_test,
                'is_train:0': False} ## for dropout
        if return_proba:
            return self.sess.run('probabilities:0',
                                 feed_dict=feed)
        else:
            return self.sess.run('labels:0',
                                 feed_dict=feed)

In [3]:
cnn=ConvNN(random_seed=123)

/var/folders/b6/sj0f93ps3rn4fdfsd4mkgj_00000gn/T/ipykernel_71924/2537897753.py:54: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  h1 = tf.layers.conv2d(tf_x_image,
/Users/apple/opt/anaconda3/envs/Deep-Learning/lib/python3.10/site-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  return layer.apply(inputs)
/var/folders/b6/sj0f93ps3rn4fdfsd4mkgj_00000gn/T/ipykernel_71924/2537897753.py:59: UserWarning: `tf.layers.max_pooling2d` is deprecated and will be removed in a future version. Please use `tf.keras.layers.MaxPooling2D` instead.
  h1_pool = tf.layers.max_pooling2d(h1,
/Users/apple/opt/anaconda3/envs/Deep-Learning/lib/python3.10/site-packages/keras/legacy_tf_layers/pooling.py:600: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



2022-05-23 19:56:46.585381: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
cnn.train(training_set=(X_train,y_train),
          validation_set=(X_valid,y_valid),
          initialize=True)
cnn.save(epoch=20)

Epoch 01: Training Avg. Loss: 308.492 Validation Acc:   0.938
Epoch 02: Training Avg. Loss:  80.949 Validation Acc:   1.000
Epoch 03: Training Avg. Loss:  53.947 Validation Acc:   1.000
Epoch 04: Training Avg. Loss:  42.224 Validation Acc:   1.000


KeyboardInterrupt: 

In [ ]:
del cnn
cnn2=ConvNN(random_seed=123)
cnn2.load(epoch=20,path='./tflayers-model/')
print(cnn2.predict(X_test[:10,:]))

In [ ]:
#计算准确率
preds=cnn2.predict(X_test)
print('Test Accuracy: %.2f%%' % (100*
      np.sum(y_test == preds)/len(y_test)))

### 本章学习了CNN或卷积神经网络，并探索了构建不同CNN体系结构所需要的模块。从定义卷积运算开始，然后通过讨论一维和二维的实现了解了它的基本原理。通过讨论最大池和平均池两种形式的池操作，我们也介绍了子采样。然后，将所有这些模块放在一起构建了深度卷积神经网络，用TensorFlow的核心API和Layers API进行了实现，并将其应用于图像识别。